In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('chromium')

2023-01-12 08:34:46,247 INFO: ====== WebDriver manager ======
2023-01-12 08:34:46,824 INFO: Get LATEST chromedriver version for chromium None
2023-01-12 08:34:47,252 INFO: Driver [C:\Users\Chris\.wdm\drivers\chromedriver\win32\109.0.5414\chromedriver.exe] found in cache


**Scrape New Completed Games and Format Them**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 0 to 15
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            16 non-null     int64         
 1   GAME_DATE_EST   16 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  16 non-null     int64         
 3   PTS             16 non-null     int64         
 4   FG_PCT          16 non-null     float64       
 5   FG3_PCT         16 non-null     float64       
 6   FT_PCT          16 non-null     float64       
 7   REB             16 non-null     int64         
 8   AST             16 non-null     int64         
 9   TEAM_ID         16 non-null     object        
 10  GAME_ID         16 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 1.5+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-11,0,97,46.6,31.0,66.7,39,16,1610612741,22200619,100,46.3,30.4,81.0,52,21,1610612764,2022
1,2023-01-11,0,118,48.9,48.5,76.2,36,22,1610612750,22200618,135,60.0,53.1,78.6,38,30,1610612765,2022
2,2023-01-11,0,114,51.7,36.7,76.5,33,22,1610612740,22200621,125,48.4,34.1,88.0,50,24,1610612738,2022
3,2023-01-11,0,113,47.8,30.8,71.4,41,27,1610612754,22200622,119,45.2,35.7,84.8,50,21,1610612752,2022
4,2023-01-11,1,114,45.9,37.5,54.5,57,28,1610612749,22200620,105,44.8,36.7,88.9,41,25,1610612737,2022
5,2023-01-11,0,129,46.7,32.4,78.3,48,33,1610612759,22200623,135,51.5,63.6,68.0,57,19,1610612763,2022
6,2023-01-11,0,97,43.3,28.1,71.4,44,28,1610612756,22200624,126,56.5,44.1,71.4,38,30,1610612743,2022
7,2023-01-11,0,115,45.8,39.0,64.7,48,30,1610612745,22200625,135,47.9,39.6,83.9,45,28,1610612758,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(driver)


print(matchups)
print(game_ids)


[['1610612760', '1610612755'], ['1610612738', '1610612751'], ['1610612749', '1610612748'], ['1610612766', '1610612761'], ['1610612742', '1610612747'], ['1610612739', '1610612757']]
['22200626', '22200627', '22200628', '22200629', '22200630', '22200631']


**Close Webdriver**

In [6]:
driver.close() 

**Create Rows for Today's Games with Empty Stats**

In [7]:
# append today's matchups to the new games dataframe


df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': int(game_ids[i]),                       
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612755,22200626,0.0,0.0,0.0,0.0,0.0,0.0,1610612760,2022
1,2023-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612751,22200627,0.0,0.0,0.0,0.0,0.0,0.0,1610612738,2022
2,2023-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612748,22200628,0.0,0.0,0.0,0.0,0.0,0.0,1610612749,2022
3,2023-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612761,22200629,0.0,0.0,0.0,0.0,0.0,0.0,1610612766,2022
4,2023-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612747,22200630,0.0,0.0,0.0,0.0,0.0,0.0,1610612742,2022
5,2023-01-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612757,22200631,0.0,0.0,0.0,0.0,0.0,0.0,1610612739,2022


**Access Feature Store**

In [8]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350
Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [9]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2023-01-12 08:36:45,819 INFO: USE `nba_predictor_featurestore`
2023-01-12 08:36:46,427 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23408,2023-01-12,22200626,1610612755,1610612760,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23409,2023-01-11,22200619,1610612741,1610612764,2022,97,46.593750,66.687500,31.000000,16,39,100,46.312500,81.000000,30.406250,21,52,0
23410,2023-01-11,22200623,1610612759,1610612763,2022,129,46.687500,78.312500,32.406250,33,48,135,51.500000,68.000000,63.593750,19,57,0
23411,2023-01-12,22200627,1610612751,1610612738,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0


**Convert Feature Names back to original mixed case**

In [11]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23408,2023-01-12,22200626,1610612755,1610612760,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23409,2023-01-11,22200619,1610612741,1610612764,2022,97,46.593750,66.687500,31.000000,16,39,100,46.312500,81.000000,30.406250,21,52,0
23410,2023-01-11,22200623,1610612759,1610612763,2022,129,46.687500,78.312500,32.406250,33,48,135,51.500000,68.000000,63.593750,19,57,0
23411,2023-01-12,22200627,1610612751,1610612738,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [12]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results

#df_old = df_old.set_index('GAME_ID')
#df_old.update(df_new.set_index('GAME_ID'))
#df_old = df_old.reset_index()  

# another approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23395,2023-01-11,22200622,1610612754,1610612752,2022,113,47.800000,71.400000,30.800000,27,41,119,45.200000,84.800000,35.700000,21,50,0
23396,2023-01-11,22200620,1610612749,1610612737,2022,114,45.900000,54.500000,37.500000,28,57,105,44.800000,88.900000,36.700000,25,41,1
23397,2023-01-11,22200623,1610612759,1610612763,2022,129,46.700000,78.300000,32.400000,33,48,135,51.500000,68.000000,63.600000,19,57,0
23398,2023-01-11,22200624,1610612756,1610612743,2022,97,43.300000,71.400000,28.100000,28,44,126,56.500000,71.400000,44.100000,30,38,0


**Add Today's Matchups for Feature Engineering**

In [13]:
df_combined = pd.concat([df_old, df_today], ignore_index = True)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0
1,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0
2,2005-11-30 00:00:00,20500210,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0
3,2009-03-15 00:00:00,20800989,1610612755,1610612748,2008,85.0,0.423096,0.666992,0.293945,24.0,40.0,77.0,0.412109,0.695801,0.262939,22.0,38.0,1.0
4,2004-11-17 00:00:00,20400109,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23401,2023-01-12,22200627,1610612751,1610612738,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23402,2023-01-12,22200628,1610612748,1610612749,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23403,2023-01-12,22200629,1610612761,1610612766,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23404,2023-01-12,22200630,1610612747,1610612742,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


**Data Processing**

In [14]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,0,1.0
1,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,0,1.0
2,2005-11-30 00:00:00,20500210,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0,0,1.0
3,2009-03-15 00:00:00,20800989,1610612755,1610612748,2008,85.0,0.423096,0.666992,0.293945,24.0,40.0,77.0,0.412109,0.695801,0.262939,22.0,38.0,1.0,0,1.0
4,2004-11-17 00:00:00,20400109,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23401,2023-01-12,22200627,1610612751,1610612738,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23402,2023-01-12,22200628,1610612748,1610612749,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23403,2023-01-12,22200629,1610612761,1610612766,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23404,2023-01-12,22200630,1610612747,1610612742,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [15]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

**Insert New Data into Feature Group**

In [16]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/23398 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2799c1fd9a0>, None)

In [17]:
df_old.tail(20)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
23380,2023-01-10,22200615,1610612756,1610612744,2022,125,45.59375,93.5000,45.187500,26,57,113,38.50000,77.8125,37.500000,24,45,1
23381,2023-01-10,22200613,1610612766,1610612761,2022,120,54.68750,73.3125,42.906250,33,34,132,49.50000,81.5000,45.500000,32,49,0
23382,2023-01-10,22200611,1610612760,1610612748,2022,111,48.40625,66.6875,25.703125,27,44,112,39.18750,100.0000,30.296875,18,41,0
23383,2023-01-10,22200617,1610612742,1610612746,2022,101,43.50000,80.6250,31.593750,15,32,113,45.31250,76.6875,38.687500,20,46,0
23384,2023-01-11,22200620,1610612749,1610612737,2022,114,45.90625,54.5000,37.500000,28,57,105,44.81250,88.8750,36.687500,25,41,1
23385,2023-01-11,22200621,1610612740,1610612738,2022,114,51.68750,76.5000,36.687500,22,33,125,48.40625,88.0000,34.093750,24,50,0
23386,2023-01-11,22200622,1610612754,1610612752,2022,113,47.81250,71.3750,30.796875,27,41,119,45.18750,84.8125,35.687500,21,50,0
23387,2023-01-11,22200625,1610612745,1610612758,2022,115,45.81250,64.6875,39.000000,30,48,135,47.90625,83.8750,39.593750,28,45,0
23388,2023-01-11,22200624,1610612756,1610612743,2022,97,43.31250,71.3750,28.093750,28,44,126,56.50000,71.3750,44.093750,30,38,0
23389,2023-01-11,22200618,1610612750,1610612765,2022,118,48.90625,76.1875,48.500000,22,36,135,60.00000,78.6250,53.093750,30,38,0


In [18]:
df_combined[df_combined['GAME_ID'] > 22200580]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
23346,2023-01-06,22200588,1610612739,1610612743,2022,108,45.50000,56.31250,38.500000,30,...,-6.220312,-5.344792,-8.000000,-7.571429,-7.5,-6.666667,2.000000,-0.428571,-1.4,-0.400000
23347,2023-01-06,22200587,1610612746,1610612750,2022,115,44.00000,66.68750,36.000000,22,...,2.623437,2.220833,-1.666667,0.857143,-0.5,-0.600000,0.333333,1.428571,3.7,5.200000
23348,2023-01-06,22200583,1610612766,1610612749,2022,138,52.00000,63.18750,41.687500,32,...,-1.440625,-1.122917,0.666667,4.285714,3.1,1.133333,-10.000000,-8.285714,-5.8,-5.333333
23349,2023-01-06,22200582,1610612752,1610612761,2022,112,47.00000,69.18750,43.187500,19,...,3.820312,6.213542,-2.000000,1.000000,1.0,0.933333,8.000000,5.857143,5.0,7.066667
23350,2023-01-06,22200581,1610612741,1610612755,2022,126,57.09375,58.81250,58.812500,33,...,1.529688,-1.044792,-7.333333,-4.428571,-2.8,-1.866667,-3.000000,2.571429,-0.5,-0.200000
23351,2023-01-06,22200586,1610612751,1610612740,2022,108,43.50000,95.00000,45.500000,26,...,5.976562,6.458333,4.666667,4.857143,2.5,1.533333,-8.333333,-7.285714,-6.4,-4.733333
23352,2023-01-06,22200585,1610612765,1610612759,2022,109,47.31250,58.81250,35.500000,23,...,-0.060938,1.584375,-4.333333,-1.571429,-3.6,-2.466667,3.000000,1.857143,1.5,2.133333
23354,2023-01-06,22200590,1610612737,1610612747,2022,114,45.81250,77.31250,29.000000,24,...,-0.782812,-1.315625,-2.333333,-1.285714,0.8,-0.733333,-4.000000,0.714286,0.6,0.133333
23355,2023-01-06,22200589,1610612748,1610612756,2022,104,42.31250,83.87500,31.593750,19,...,-7.257812,-3.931250,2.000000,-1.285714,-0.7,-1.333333,0.333333,-0.857143,-2.1,-2.933333
23356,2023-01-06,22200584,1610612764,1610612760,2022,110,48.09375,75.00000,29.203125,21,...,-5.698438,-2.576042,6.333333,4.000000,1.6,1.733333,8.666667,2.571429,1.2,-1.133333
